In [1]:
import requests as req
from datetime import datetime
import pandas as pd

from bs4 import BeautifulSoup as BS
from connection.var import ConnectionVar_forex, ConnectionVar_crypto, agent_desktop, parser, URL

In [14]:
df_columns = ['DATE','PRICE','PRICE_CAP', 'VOLUME_24','CIRCULATION', 'PERCENT_CHG']
data_columns = ['NAME','PRICE','PRICE_CAP', 'VOLUME_24','CIRCULATION', 'PERCENT_CHG']
tag_names = 'a'


def get_html(url, header):
    #get data from sources
    #url for various sources
    connect = req.get(url, header)
    data = connect.content
    return data

def scrape_data_from_web(where, tag, id_name, url_id):

    conn = get_html(URL[url_id], header=agent_desktop)
    soup = BS(conn, parser)

    date_as = datetime.strftime(datetime.now(), '%m-%d-%Y, %H:%M')
    scraped_data = []
    
    if where == 'crypto':
        
        for t, n in tag, id_name:
            scraped_data.append([x.get_text() for x in soup.find_all(t, class_=ConnectionVar_crypto[n])])
    
        df = pd.DataFrame(index=scraped_data[0], columns=df_columns)

        df['DATE'] = date_as
        df['PRICE'] = scraped_data[1]
        df['PRICE_CAP'] = scraped_data[2]
        df['VOLUME_24'] = scraped_data[3]
        df['CIRCULATION'] = scraped_data[4]
        df['PERCENT_CHG'] = scraped_data[5]

        #df.to_sql(sqlite_table, sqlite_conn, if_exists='append')
        return df.head(19)
    
    elif where == 'forex':
        
        scraped_data = ([x.get_text() for x in soup.find_all(tag, class_=ConnectionVar_forex[id_name])])

        #df.to_sql(sqlite_table, sqlite_conn, if_exists='append')
        return scraped_data


In [15]:
scrape_data_from_web(where='forex', tag='td', url_id=0, id_name='PRICE')

[]